# WEBSCRAPING MOVIE RELEASE DATES IN GERMANY

### IMPORT LIBRARIES

In [20]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pprint import pprint
from lxml import html
from lxml.html import fromstring
import urllib.request
from urllib.request import urlopen
import random
import re
import time
import subprocess
from datetime import datetime
import urllib.request
import unidecode
#import scrapy

### SCRAPING TARGET

In [21]:
#URL KINO-ZEIT.DE CINEMA PROGRAM
url='https://www.kino-zeit.de/filme/filmstarts/aktuell'

### SETTING SCOPE
In Germany theatrical releases are typically on Thursdays.
Setting 'weeks' to 1 will return last Thursdays theatrical releases.
Setting 'weeks' to 2 will return last Thursdays and upcoming Thursday's theatrical releases.

In [22]:
#CHOOSING NUMBER OF UPCOMING WEEKS TO SCRAPE
print('How many upcoming weeks would you like to scrape at kino-zeit.de\n---')
weeks=int(input())

How many upcoming weeks would you like to scrape at kino-zeit.de
---
8


### FUNCTIONS

In [23]:
#FUNCTION TO TURN URL INTO HTML
def make_soup(url):
    html=requests.get(url).content
    return BeautifulSoup(html,'html.parser')

#FUNCTION CREATING THE URL TO FLIP TO NEXT WEEKS PAGE
def next_url(soup):
    url=''.join([f'https://www.kino-zeit.de/filme/filmstarts/{i["href"]}' for i in soup.find_all("a",{"class": "next btn btn-primary"})])
    return url

#FUNCTION FLIPPING THROUGH THE WEEKLY OVERVIEWS
#Picking film titles, start dates and the film's detail page
def how_much_soup(url, weeks):
    df_loop=pd.DataFrame(columns=['start','film','director','actor','link','image','distro','country','trailer'])
    lst_start=[]
    lst_film=[]
    lst_link=[]

    for x in range(weeks): #for the number of weeks we are browsing
        
        soup= make_soup(url)
        lst_film.append([i.text for i in soup.find_all(['h3','a'],{'class': 'filmlink'})])
        lst_start.append([i.text for i in soup.find_all('div',{'class':'col-md-2 infospalte-1'})])
        lst_start=re.findall(r'\d\d.\d\d.\d\d\d\d', str(lst_start))
        lst_link.append([f'https://www.kino-zeit.de{i["href"]}' for i in soup.find_all(["h3","a"],{"class": "filmlink"})])
        url=next_url(soup) #calling the function for the url we flip to for the following week
    
    lst_film=[i for sublst in lst_film for i in sublst]
    df_loop['film']=lst_film
    df_loop['year']=df_loop['film'].str.findall('\d\d\d\d')
    df_loop['year']=df_loop['year'].apply(listToString)
    #df_loop['start']=lst_start
    lst_start=[datetime.strptime(i,'%d.%m.%Y') for i in lst_start]
    df_loop['start']=lst_start
    lst_link= [i for sublst in lst_link for i in sublst]
    df_loop['link']=lst_link
    return (df_loop)

In [24]:
# FUNCTION TO TURN LIST TO STRING
#NEEDED FOR CLEANING UP ACTORS' COLUMN
def listToString(s):  
    
    # initialize an empty string 
    str1 = " " 
    
    # return string   
    return (str1.join(s)) 

In [25]:
#FUNCTION GETTING INFOR FORM FILM DETAIL PAGES
def filmdetails(df, link):
    #CREATING OVERVIEW
    regie_list=[]
    darsteller_list=[]
    list_img_url=[]
    list_verleih=[]
    list_prodland=[]
    
    #BROWSING THE FILMS' INDIVIDUAL PAGES FOR FURTHER DETAILS 
    for i in link:
        print(i)
        sub_page=make_soup(i)
        regie=[i.text.replace('\nRegie','').replace('\n\n','').split(',') for i in sub_page.find_all('div',{'class':'field field--name-field-regie field--type-entity-reference field--label-above'})]
        regie_list.append(regie[0])
        darsteller=[i.text.replace('\nDarsteller','').split('\n') for i in sub_page.find_all('div',{'class':'field field--name-field-hauptdarsteller field--type-entity-reference field--label-above'})]
        darsteller_list.append(darsteller)
        img_url=[i['data-big'].split('?') for i in sub_page.find_all('img',{'id':'headerbildbig'})]
        list_img_url.append(i[0] for i in img_url)
        verleih=[i.text.replace('\nFilmverleih','').replace('\n\n','') for i in sub_page.find_all(['div','a'],{'class':'field field--name-field-distribution-film field--type-entity-reference field--label-above'})]
        list_verleih.append(verleih)
        prodland=[i.text.replace('\nProduktionsland','').replace('\n\n','') for i in sub_page.find_all(['div','a'],{'class':'field field--name-field-produktionsland field--type-entity-reference field--label-above'})]
        list_prodland.append(prodland)

    #FILLING THE DATAFRAME WITH THOSE NEW DATA
    df['director']=regie_list
    df['actor']=darsteller_list
    df['image']=list_img_url
    df['distro']=list_verleih
    df['country']=list_prodland
    
    #CLEAN ACTORS COLUMN
    a=[]
    for i in range(len(df.actor)):
        if len(df['actor'].loc[i])==2:
            a.append(list(filter(None,df.actor[i][0])))
        else:
            a.append([''])
        
    df['actor']=a
    df['actor']=df['actor'].apply(listToString)

    #TURN LISTS INTO STRINGS TO CLEAN THEM UP
    for i in range(len(df)):
        
        df['film']=df['film'].str.replace('\(\d\d\d\d\)','')
        df['country'][i]=''.join(df['country'][i]).replace('\n',', ')
        df['director'][i]=', '.join(df['director'][i]).replace('\n',', ')
        df['image'][i]=''.join(df['image'][i])
        df['distro'][i]=''.join(df['distro'][i]).replace('\n',', ')
    return df

In [26]:
### DEALING WITH UMLAUT CHARACTERS
### CURRENTLY NOT IN USE
"""def remove_odd_characters(string):
    
    u = 'ü'.encode()
    U = 'Ü'.encode()
    a = 'ä'.encode()
    A = 'Ä'.encode()
    o = 'ö'.encode()
    O = 'Ö'.encode()
    ss = 'ß'.encode()
    e = 'é' .encode()
    e = 'è' .encode()

    string = string.encode()
    string = string.replace(u, b'ue')
    string = string.replace(U, b'Ue')
    string = string.replace(a, b'ae')
    string = string.replace(A, b'Ae')
    string = string.replace(o, b'oe')
    string = string.replace(O, b'Oe')
    string = string.replace(ss, b'ss')
    string = string.replace(e, b'e')

    string = string.decode('utf-8')
    return string"""

"def remove_odd_characters(string):\n    \n    u = 'ü'.encode()\n    U = 'Ü'.encode()\n    a = 'ä'.encode()\n    A = 'Ä'.encode()\n    o = 'ö'.encode()\n    O = 'Ö'.encode()\n    ss = 'ß'.encode()\n    e = 'é' .encode()\n    e = 'è' .encode()\n\n    string = string.encode()\n    string = string.replace(u, b'ue')\n    string = string.replace(U, b'Ue')\n    string = string.replace(a, b'ae')\n    string = string.replace(A, b'Ae')\n    string = string.replace(o, b'oe')\n    string = string.replace(O, b'Oe')\n    string = string.replace(ss, b'ss')\n    string = string.replace(e, b'e')\n\n    string = string.decode('utf-8')\n    return string"

In [27]:
### DEALING WITH FOREIGN LANGUAGE CHARACTERS

def recoding_string (df):
    trailer_list=[]
    for i in range(len(df)):
        search= f"{df['film'][i].replace(' ','_')}_trailer_{df['distro'][i].replace(' ','_')}"
        #search=remove_odd_characters(search) --- calls function remove umlaut
        search=unidecode.unidecode(search) # dealing with foreign language letters so we can include them in youtube search URL
        print(search)
        html = urllib.request.urlopen(f"https://www.youtube.com/results?search_query={search}")
        video_ids = re.findall(r"watch\?v=(\S{11})", html.read().decode())
        if video_ids == []:
            trailer_list.append('no trailer found')
        else:
            youtube_url= f'https://youtu.be/{video_ids[0]}'
            trailer_list.append(youtube_url)
    return trailer_list

# START PROGRAM

In [28]:
#CALL FUNCTION FILM OVERVIEW
df=how_much_soup(url, weeks)
df


,start,film,director,actor,link,image,distro,country,trailer,year
0,2021-12-02,Aksel - The Story of Aksel Lund Svindal (2021),NaN,NaN,https://www.kino-zeit.de/node/53279,NaN,NaN,NaN,NaN,2021
1,2021-12-02,Aykut Eniste 2 (2021),NaN,NaN,https://www.kino-zeit.de/node/53493,NaN,NaN,NaN,NaN,2021
2,2021-12-02,Benedetta (2021),NaN,NaN,https://www.kino-zeit.de/node/44691,NaN,NaN,NaN,NaN,2021
3,2021-12-02,"Bloody Nose, Empty Pockets (2020)",NaN,NaN,https://www.kino-zeit.de/node/48502,NaN,NaN,NaN,NaN,2020
4,2021-12-02,Clifford - Der große rote Hund (2021),NaN,NaN,https://www.kino-zeit.de/node/52709,NaN,NaN,NaN,NaN,2021
...,...,...,...,...,...,...,...,...,...,...
59,2022-01-20,Effigie - Das Gift und die Stadt (2019),NaN,NaN,https://www.kino-zeit.de/node/53246,NaN,NaN,NaN,NaN,2019
60,2022-01-20,Eine Nacht in Helsinki (2020),NaN,NaN,https://www.kino-zeit.de/node/52819,NaN,NaN,NaN,NaN,2020
61,2022-01-20,Licorice Pizza (2021),NaN,NaN,https://www.kino-zeit.de/node/53184,NaN,NaN,NaN,NaN,2021
62,2022-01-20,Niemand ist bei den Kälbern (2021),NaN,NaN,https://www.kino-zeit.de/node/51792,NaN,NaN,NaN,NaN,2021


In [29]:
# Spezical Fall Film dem zB der Regie Credit fehlt erzeugt list our of range fehler. 
#Die entsprechenden Filme muss man entfernen.
df= df[df['film']!='SimsalaGrimm (2020)'].reset_index(drop=True)


In [30]:
#CALL FUNCTION FILM DETAILS
df=filmdetails(df, df['link'])
df

https://www.kino-zeit.de/node/53279
https://www.kino-zeit.de/node/53493
https://www.kino-zeit.de/node/44691
https://www.kino-zeit.de/node/48502
https://www.kino-zeit.de/node/52709
https://www.kino-zeit.de/node/50248
https://www.kino-zeit.de/node/52936
https://www.kino-zeit.de/node/51954
https://www.kino-zeit.de/node/51136
https://www.kino-zeit.de/node/53550
https://www.kino-zeit.de/node/51978
https://www.kino-zeit.de/node/53555
https://www.kino-zeit.de/node/50368
https://www.kino-zeit.de/node/50613
https://www.kino-zeit.de/node/46415
https://www.kino-zeit.de/node/53500
https://www.kino-zeit.de/node/53056
https://www.kino-zeit.de/node/53450
https://www.kino-zeit.de/node/53472
https://www.kino-zeit.de/node/47235
https://www.kino-zeit.de/node/51012
https://www.kino-zeit.de/node/50048
https://www.kino-zeit.de/node/53147
https://www.kino-zeit.de/node/51221
https://www.kino-zeit.de/node/52982
https://www.kino-zeit.de/node/49068
https://www.kino-zeit.de/node/45110
https://www.kino-zeit.de/nod

<ipython-input-25-a815aaeac6f9>:46: FutureWarning: The default value of regex will change from True to False in a future version.
  df['film']=df['film'].str.replace('\(\d\d\d\d\)','')
<ipython-input-25-a815aaeac6f9>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['country'][i]=''.join(df['country'][i]).replace('\n',', ')
<ipython-input-25-a815aaeac6f9>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['director'][i]=', '.join(df['director'][i]).replace('\n',', ')
<ipython-input-25-a815aaeac6f9>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in th

,start,film,director,actor,link,image,distro,country,trailer,year
0,2021-12-02,Aksel - The Story of Aksel Lund Svindal,"Filip Christensen, Even Sigstad",,https://www.kino-zeit.de/node/53279,https://www.kino-zeit.de/sites/default/files/s...,Panda Lichtspiele Filmverleih,Norwegen,NaN,2021
1,2021-12-02,Aykut Eniste 2,Onur Bilgetay,Müfit Kayacan Melis Babadag Cem Gelinoglu Neza...,https://www.kino-zeit.de/node/53493,https://www.kino-zeit.de/sites/default/files/s...,Cinedex OG,Türkei,NaN,2021
2,2021-12-02,Benedetta,Paul Verhoeven,Charlotte Rampling Lambert Wilson Virginie Efi...,https://www.kino-zeit.de/node/44691,https://www.kino-zeit.de/sites/default/files/s...,"Capelight Pictures, Koch Films GmbH, Central Film",Frankreich,NaN,2021
3,2021-12-02,"Bloody Nose, Empty Pockets","Bill Ross, Turner Ross",,https://www.kino-zeit.de/node/48502,https://www.kino-zeit.de/sites/default/files/s...,UCM.ONE,USA,NaN,2020
4,2021-12-02,Clifford - Der große rote Hund,Walt Becker,Darby Camp John Cleese Sienna Guillory Jack Wh...,https://www.kino-zeit.de/node/52709,https://www.kino-zeit.de/sites/default/files/s...,Paramount Pictures Germany,"Großbritannien, Kanada, USA",NaN,2021
...,...,...,...,...,...,...,...,...,...,...
59,2022-01-20,Effigie - Das Gift und die Stadt,Udo Flohr,Suzan Anbeh Elisa Thiemann Christoph Gottschal...,https://www.kino-zeit.de/node/53246,https://www.kino-zeit.de/sites/default/files/s...,Filmdisposition Wessel,Deutschland,NaN,2019
60,2022-01-20,Eine Nacht in Helsinki,Mika Kaurismäki,Kari Heiskanen Anu Sinisalo Pertti Sveholm Tim...,https://www.kino-zeit.de/node/52819,https://www.kino-zeit.de/sites/default/files/s...,,Finnland,NaN,2020
61,2022-01-20,Licorice Pizza,Paul Thomas Anderson,Cooper Hoffman Alana Haim Bradley Cooper Sean ...,https://www.kino-zeit.de/node/53184,https://www.kino-zeit.de/sites/default/files/s...,Universal Pictures International Germany GmbH,USA,NaN,2021
62,2022-01-20,Niemand ist bei den Kälbern,Sabrina Sarabi,Saskia Rosendahl Rick Okon Enno Trebs Elisa Sc...,https://www.kino-zeit.de/node/51792,https://www.kino-zeit.de/sites/default/files/s...,Filmwelt Verleihagentur,Deutschland,NaN,2021


In [31]:
df['trailer']=recoding_string(df)

Aksel_-_The_Story_of_Aksel_Lund_Svindal__trailer_Panda_Lichtspiele_Filmverleih
Aykut_Eniste_2__trailer_Cinedex_OG
Benedetta__trailer_Capelight_Pictures,_Koch_Films_GmbH,_Central_Film
Bloody_Nose,_Empty_Pockets__trailer_UCM.ONE
Clifford_-_Der_grosse_rote_Hund__trailer_Paramount_Pictures_Germany_
Das_Ende_des_Schweigens__trailer_GMfilms,_barnsteiner-film_
Die_Hand_Gottes__trailer_Netflix,_Filmwelt_Verleihagentur
Die_Zahmung_der_Baume_-_Taming_the_Garden__trailer_Film_Kino_Text
Gunpowder_Milkshake__trailer_STUDIOCANAL_GmbH_Filmverleih
Harald_Naegeli_-_Der_Sprayer_von_Zurich__trailer_missingfilms
House_of_Gucci__trailer_Universal_Pictures_International_Germany
Toys_of_Terror__trailer_Warner_Bros._Entertainment_GmbH_
Vater_-_Otac__trailer_barnsteiner_Film
Weihnachten_im_Zaubereulenwald__trailer_justbridge_entertainment/24_Bilder_
Adam__trailer_Grandfilm
Das_grosse_Abenteuer_des_kleinen_Vampir__trailer_STUDIOCANAL_GmbH_Filmverleih
Don't_Look_Up__trailer_Netflix,_Filmwelt_Verleihagentur
La_Li

In [32]:
df

,start,film,director,actor,link,image,distro,country,trailer,year
0,2021-12-02,Aksel - The Story of Aksel Lund Svindal,"Filip Christensen, Even Sigstad",,https://www.kino-zeit.de/node/53279,https://www.kino-zeit.de/sites/default/files/s...,Panda Lichtspiele Filmverleih,Norwegen,https://youtu.be/mJa3fa9243A,2021
1,2021-12-02,Aykut Eniste 2,Onur Bilgetay,Müfit Kayacan Melis Babadag Cem Gelinoglu Neza...,https://www.kino-zeit.de/node/53493,https://www.kino-zeit.de/sites/default/files/s...,Cinedex OG,Türkei,https://youtu.be/QmC7t82Ectk,2021
2,2021-12-02,Benedetta,Paul Verhoeven,Charlotte Rampling Lambert Wilson Virginie Efi...,https://www.kino-zeit.de/node/44691,https://www.kino-zeit.de/sites/default/files/s...,"Capelight Pictures, Koch Films GmbH, Central Film",Frankreich,https://youtu.be/y0O9QfUOkOY,2021
3,2021-12-02,"Bloody Nose, Empty Pockets","Bill Ross, Turner Ross",,https://www.kino-zeit.de/node/48502,https://www.kino-zeit.de/sites/default/files/s...,UCM.ONE,USA,https://youtu.be/6oeNwBek15A,2020
4,2021-12-02,Clifford - Der große rote Hund,Walt Becker,Darby Camp John Cleese Sienna Guillory Jack Wh...,https://www.kino-zeit.de/node/52709,https://www.kino-zeit.de/sites/default/files/s...,Paramount Pictures Germany,"Großbritannien, Kanada, USA",https://youtu.be/3GXfzl8tIcY,2021
...,...,...,...,...,...,...,...,...,...,...
59,2022-01-20,Effigie - Das Gift und die Stadt,Udo Flohr,Suzan Anbeh Elisa Thiemann Christoph Gottschal...,https://www.kino-zeit.de/node/53246,https://www.kino-zeit.de/sites/default/files/s...,Filmdisposition Wessel,Deutschland,https://youtu.be/w4fGHlf2Z5I,2019
60,2022-01-20,Eine Nacht in Helsinki,Mika Kaurismäki,Kari Heiskanen Anu Sinisalo Pertti Sveholm Tim...,https://www.kino-zeit.de/node/52819,https://www.kino-zeit.de/sites/default/files/s...,,Finnland,https://youtu.be/W3z8sZbrLD8,2020
61,2022-01-20,Licorice Pizza,Paul Thomas Anderson,Cooper Hoffman Alana Haim Bradley Cooper Sean ...,https://www.kino-zeit.de/node/53184,https://www.kino-zeit.de/sites/default/files/s...,Universal Pictures International Germany GmbH,USA,https://youtu.be/mdN3tA6sxFY,2021
62,2022-01-20,Niemand ist bei den Kälbern,Sabrina Sarabi,Saskia Rosendahl Rick Okon Enno Trebs Elisa Sc...,https://www.kino-zeit.de/node/51792,https://www.kino-zeit.de/sites/default/files/s...,Filmwelt Verleihagentur,Deutschland,https://youtu.be/pLqh6jbLBHo,2021


### Subsetting 
films with production country 'Deutschland' and after a certain date

In [33]:
df=df[(df['country'].str.contains('Deutschland'))&(df['start']>'2021-12-14')]


## EXPORT EXCEL + CSV  WITH TIMESTAMP

In [34]:
timestr = time.strftime("%Y%m%d")
df.to_csv(f'exported_files/Filmstarts_{timestr}.csv')
df.to_excel(f'exported_files/Filmstarts_{timestr}.xlsx')

### Option download images

In [35]:
#def img_download(filmtitle, url):
for i in range(len(df)):
    response = requests.get(df.iloc[i][5])
    file = open(f'exported_files/{df.iloc[i][1]}.jpg', 'wb')
    file.write(response.content)
    file.close()

### Option to have resulting Excel sent by email

In [36]:
import os
import smtplib
from email.message import EmailMessage

email_address=os.environ.get('email_user')
email_password=os.environ.get('email_pass')
smtp_ssl_host=os.environ.get('DFA_mailserver')
smtp_ssl_port = 465

msg= EmailMessage()
msg['Subject']= 'Filmstarts sind exportiert! - dein bot'
msg['From']=email_address
msg['To']='spitzer@deutsche-filmakademie.de'
msg.set_content('Hallo, hallo, \n\nHier ein Mailtest aus Jupyter Notebook.\n\nDas Excel mit den aktuellen Filmstarts ist anbei. Gefiltert nach Filmen mit Produktionsland Deutschland. Und Filmstart nach dem 5.November. Die Fotos sind in der Mediathek\n\nschöne Grüße dein Python Bot!')

with open(f'exported_files/Filmstarts_{timestr}.xlsx', 'rb') as attachment:
    file=attachment.read()

msg.add_attachment(file,maintype='Excel',subtype='xlsx', filename=f'Filmstarts_{timestr}.xlsx')

with smtplib.SMTP_SSL(smtp_ssl_host, smtp_ssl_port) as server:
    server.login(email_address, email_password)
    server.send_message(msg)
    server.quit()

### RINGS BELL WHEN SCRIPT FINISHED

In [37]:
#RINGS THE BELL WHEN SCRIPT IS DONE
audio_file = "service-bell.mp3"
return_code = subprocess.call(["afplay", audio_file])

In [38]:
"""
a=[]
for i in range(len(df.actor)):
        if len(df['actor'].loc[i])==2:
            a.append(list(filter(None,df.actor[i][0])))
        else:
           a.append([''])
df['actor_test']=a
#df['actor_test']=listToString(list(df['actor_test']))
df['actor_test']=df['actor_test'].apply(listToString)
df
"""

"\na=[]\nfor i in range(len(df.actor)):\n        if len(df['actor'].loc[i])==2:\n            a.append(list(filter(None,df.actor[i][0])))\n        else:\n           a.append([''])\ndf['actor_test']=a\n#df['actor_test']=listToString(list(df['actor_test']))\ndf['actor_test']=df['actor_test'].apply(listToString)\ndf\n"